In [140]:
%matplotlib qt4
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style("ticks", {"legend.frameon": True})

### Learning Curve

In [154]:
points = np.arange(0, 10, 0.001)
func = lambda x: -100/(x + 1) + 100

plt.figure(num=None, figsize=(4, 3), dpi=160)
plt.plot(points, [func(p) for p in points])
plt.xlabel('time')
plt.ylabel('memory strength')
plt.tight_layout()
sns.despine()

### Forgetting Curve

In [130]:
points = np.arange(0, 10, 0.001)
func_forget = lambda x: 100/(x + 1)

plt.figure(num=None, figsize=(4, 3), dpi=160)
plt.plot(points, [func_forget(p) for p in points])
plt.xlabel('time')
plt.ylabel('memory strength')
plt.tight_layout()
sns.despine()